In [ ]:
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt


# Ruta local al archivo (ajusta la ruta según tu sistema)
ruta_archivo = 'C:/Users/Acer/OneDrive/Escritorio/primer_parcial_ia/Temp_Asu20092021.xlsx'

# Leer el archivo CSV y crear un DataFrame
# Cargar el dataset
data = pd.read_csv("Temp_Asu20092021.csv")

# Convertir la columna 'Fecha' a tipo datetime
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Establecer la columna 'Fecha' como índice
data.set_index('Fecha', inplace=True)

# Filtrar datos desde 2019 en adelante
data = data[data.index.year >= 2019]

# Eliminar filas con valores faltantes
data.dropna(inplace=True)
#. Reindexar el DataFrame para tener una frecuencia horaria

data = data.resample('H').interpolate()
# Resamplear el conjunto de datos a intervalos diarios y calcular la temperatura máxima diaria
daily_max_temperatures = data['Temperatura'].resample('D').max()

data['Max_Temperature_Day'] = data.groupby(data.index.date)['Temperatura'].transform('max')
# . Calcular el percentil 95 de la temperatura diaria
data['Percentil_95'] = data.groupby(data.index.date)['Temperatura'].transform(lambda x: x.quantile(0.95))


# ----> EXPLORACIÓN DE CARACTERÍSTICAS <----

# Calcular la matriz de correlación
correlation_matrix = data.corr()

# Visualizar la matriz de correlación con un mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()
# Objetivo: predecir la temperatura máxima del siguiente día
data['NextDay_Temp'] = data['Max_Temperature_Day'].shift(-24)

# Eliminar filas con valores faltantes tras el desplazamiento
data.dropna(inplace=True)

# Definir características (features) y variable objetivo (target)
X = data[['Temperatura']]  # Puedes agregar más características según sea necesario
y = data['NextDay_Temp']

# Normalizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y validación
train_indices = (data.index.year <= 2020)
val_indices = (data.index.year == 2021)

X_train, X_val = X_scaled[train_indices], X_scaled[val_indices]
y_train, y_val = y[train_indices], y[val_indices]

# Definir el modelo de regresión Ridge
ridge = Ridge()

# Definir la cuadrícula de parámetros para GridSearch
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]  # Regularización L2
}

# Usar GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de validación
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
(f"Mean Squared Error en validación: {mse}")
#  Mostrar las primeras filas del DataFrame
# ----> INICIO DEL CÁLCULO DE MÉTRICAS DE EVALUACIÓN <----


mse = mean_squared_error(y_val, y_pred)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')

# ----> FIN DEL CÁLCULO DE MÉTRICAS DE EVALUACIÓN <----
# ----> INICIO DEL CÓDIGO DE GRIDSEARCHCV <---- 

# Definir el modelo de regresión Ridge
model = Ridge()

# Definir la cuadrícula de hiperparámetros
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Realizar Grid Search con validación cruzada de 5 pliegues
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print(f'Mejores hiperparámetros: {best_params}')

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# ----> FIN DEL CÓDIGO DE GRIDSEARCHCV <----

# ----> FIN DE LA CONSTRUCCIÓN Y EVALUACIÓN DEL MODELO FINAL <----

print(data.head())
# ----> GRÁFICO DE DISPERSIÓN <----

plt.figure(figsize=(8, 6))
plt.scatter(y_val, y_pred) # Asegúrate de usar y_val aquí
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Gráfico de dispersión: Valor real vs. Predicción')

# Agregar una línea diagonal (para una referencia visual)
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], color='red', linestyle='--') 

plt.show()

# ----> FIN DEL GRÁFICO DE DISPERSIÓN <----



: 

: 